In [1]:
import numpy as np
train_x = np.load("train_x.npy")
train_y = np.load("train_y.npy")
test_x = np.load("test_x.npy")
test_y = np.load("test_y.npy")
import tensorflow as tf

In [2]:
from scipy import misc
train_x = train_x.reshape(60 * 8000, 101, 101)
test_x = test_x.reshape(60 * 2000, 101, 101)

In [3]:
new_train_x = []
for i in range(60 * 8000):
    new_train_x.append(misc.imresize(train_x[i], (128, 128)))

new_test_x = []
for i in range(60 * 2000):
    new_test_x.append(misc.imresize(test_x[i], (128, 128)))

In [4]:
train_x = np.asarray(new_train_x, dtype=np.uint8)
test_x = np.asarray(new_test_x, dtype=np.uint8)

In [9]:
train_x.reshape(8000 * 60, 128, 128)
test_x.reshape(2000 * 60, 128, 128)
train_x = np.asarray(train_x, dtype=np.uint8)
test_x = np.asarray(test_x, dtype=np.uint8)
train_x.shape, test_x.shape

((480000, 128, 128), (120000, 128, 128))

In [10]:
train_y = np.repeat(train_y, 60)
test_y = np.repeat(test_y, 60)
train_y = np.apply_along_axis(lambda x: 1 if x >= 20 else 0, 1, train_y.reshape(60 * 8000, 1))
test_y = np.apply_along_axis(lambda x: 1 if x >= 20 else 0, 1, test_y.reshape(60 * 2000, 1))
train_y = np.repeat(train_y, 2)
test_y = np.repeat(test_y, 2)

for i in range(0, 60 * 8000):
    if (train_y[2 * i] == 0):
        train_y[2 * i] = 1
    else:
        train_y[2 * i] = 0

for i in range(0, 60 * 2000):
    if (test_y[2 * i] == 0):
        test_y[2 * i] = 1
    else:
        test_y[2 * i] = 0

In [13]:
train_y = np.asarray(train_y, dtype=np.float32).reshape(60 * 8000, 2)
test_y = np.asarray(test_y, dtype=np.float32).reshape(60 * 2000, 2)

In [14]:
train_y.shape, test_y.shape

((480000, 2), (120000, 2))

In [15]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config)

In [35]:
n_input = 128 * 128
x = tf.placeholder(tf.uint8, shape=[None, 128, 128], name="x")
y_ = tf.placeholder(tf.float32, shape=[None, 2], name="y_")

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W ,strides=[1, 1, 1 ,1], padding="SAME")
def max_pool_4x4(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                         strides=[1, 4, 4, 1], padding="SAME")

# First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 128, 128, 1])
float_x = tf.to_float(x_image)
h_conv1 = tf.nn.relu(conv2d(float_x, W_conv1) + b_conv1)
h_pool1 = max_pool_4x4(h_conv1)

# Secondary Convolution Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_4x4(h_conv2)

# Densely Connected Layer
W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8 * 8 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32, name="drop")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Read Layer
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [36]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y, random_state = 0)
test_x, test_y = shuffle(test_x, test_y, random_state = 0)

In [37]:
sess.run(tf.global_variables_initializer())

In [ ]:
import sys
train_size = 40
test_size = 100
def get_batch_train(x, y, i, train_size):
    return x[i * train_size: (i + 1) * train_size], y[i * train_size: (i + 1) * train_size]

def get_batch_test(x, y, i, test_size):
    return x[i * test_size: (i + 1) * test_size], y[i * test_size: (i + 1) * test_size]

total_train_iteration = len(train_x) / train_size
for i in range(4000):
    for train_batch_step in range(int(total_train_iteration)):
        if train_batch_step % 400 == 0:
            str_log = ("%.2f %% " % (train_batch_step * 100 / total_train_iteration))
            sys.stdout.write('\r' + str_log)
        batch_x, batch_y = get_batch_train(train_x, train_y, train_batch_step, train_size)
        sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
    
    total_iteration = len(test_x) / test_size
    total_accuracy = 0.0
    print("start evaluate")
    for test_batch_step in range(int(total_iteration)):
        test_batch_x, test_batch_y = get_batch_test(test_x, test_y, test_batch_step, test_size)
        total_accuracy += accuracy.eval(feed_dict={
            x:test_batch_x, y_: test_batch_y, keep_prob: 1.0}, session=sess)
    
    print("step %d, training accuracy %g"%(i, total_accuracy / total_iteration))

40.00 % 

In [ ]:
y_result = tf.nn.softmax(y_conv)
test = sess.run(y_result, feed_dict = {x: train_x[110:130], keep_prob: 0.5})

In [ ]:
saver = tf.train.Saver()
save_path = saver.save(sess, "my_model/4_level_cnn/54step.ckpt")

In [ ]:
import numpy as np
train_x = np.load("data/single/train_x.npy")
train_y = np.load("data/single/train_y.npy")
test_x = np.load("data/single/test_x.npy")
test_y = np.load("data/single/test_y.npy")
import tensorflow as tf

In [ ]:
feature_x_train = []
total_train_iteration = int(len(train_x) / 60)
for i in range(total_train_iteration):
    result = sess.run(h_fc1, feed_dict={x:train_x[i * 60 : (i + 1) * 60]})
    feature_x_train.append(result)
    
total_test_iteration = int(len(test_x) / 60)
for i in range(total_test_iteration):
    result = sess.run(h_fc1, feed_dict={x:test_x[i * 60 : (i + 1) * 60]})
    feature_x_train.append(result)


In [ ]:
feature = sess.run(h_fc1, feed_dict={x: train_x[0:1]})

In [ ]:
feature_x_train = np.assarray(feature_x_train).reshape(10000, 60 * 1024)

In [ ]:
len(feature[0])

In [ ]:
total_train_iteration

In [ ]:
print(train_x[0 * 60 : (0 + 1) * 60])